In [132]:
import sys
import re
import os
from collections import Counter

import numpy as np
import pandas as pd
from pathlib import Path

## Generate RNA seq DataFrame

The tsv file containing the RNA sequencing results are parsed, cleaned and then restricted based on parameters provided by the user. A DataFrame containing the parsed RNA seq data is returned.

In [133]:
# Store variables for processing of RNA seq and SILAC files into DataFrames
class ArgBin:
    def __init__(
                    self, rna_in_path_, prot_in_path_, pair_in_path_, rna_out_path_=Path(Path.cwd() / "rna_df.xlsx"), 
                    prot_out_path_=Path(Path.cwd() / "prot_df.xlsx"), comb_out_path_=Path(Path.cwd() / "comb_df.xlsx"), 
                    sample_=None, terms_=None, search_locs_="nr_description", universal_=False, dump_=False, restrict_=False
                ):
        
        self.rna_in_path = rna_in_path_
        self.prot_in_path = prot_in_path_
        self.pair_in_path = pair_in_path_
        self.rna_out_path = rna_out_path_
        self.prot_out_path = prot_out_path_
        self.comb_out_path = comb_out_path_
        self.sample = sample_
        self.terms = [] if terms_ is None else terms_
        self.search_locs = search_locs_
        self.universal = universal_
        self.dump = dump_
        self.restrict = restrict_

In [134]:
def clean_rna_df(df, args):
    # Drop redundant column
    df.drop("transcript_id(s)", axis=1, inplace=True)

    # Rename columns where helpful
    rename_dict = {
                    "Cellular Components": "cell_component", "Molecular Function": "mol_func", 
                    "Biological Process": "bio_process", "Kegg Orthology": "kegg_orth",
                    "Nr Description": "nr_description"
                  }
    df.rename(columns=rename_dict, inplace=True)

    # Fill empty cells with placeholder
    df.fillna("NaN", inplace=True)

    # Take sample of input data if requested 
    if args.sample:
        df = df.sample(min(df.shape[0], args.sample))

    # Add mean and standard deviation columns
    df["AS_mean"] = df[["AS_1_FPKM", "AS_2_FPKM", "AS_3_FPKM"]].mean(axis=1)
    df["AS_stdev"] = df[["AS_1_FPKM", "AS_2_FPKM", "AS_3_FPKM"]].std(axis=1)
    df["NS_mean"] = df[["NS_1_FPKM", "NS_2_FPKM", "NS_3_FPKM"]].mean(axis=1)
    df["NS_stdev"] = df[["NS_1_FPKM", "NS_2_FPKM", "NS_3_FPKM"]].std(axis=1)
    df["YPD_mean"] = df[["YPD_1_FPKM", "YPD_2_FPKM", "YPD_3_FPKM"]].mean(axis=1)
    df["YPD_stdev"] = df[["YPD_1_FPKM", "YPD_2_FPKM", "YPD_3_FPKM"]].std(axis=1)

    # Add log2-fold change columns
    YPD_log2 = np.log2(df["YPD_mean"] + 1)
    NS_log2 = np.log2(df["NS_mean"] + 1)
    AS_log2 = np.log2(df["AS_mean"] + 1)
    
    df["YN_L2FC"] = NS_log2 - YPD_log2
    df["YA_L2FC"] = AS_log2 - YPD_log2
    df["AN_L2FC"] = AS_log2 - NS_log2

    # Add gene_name column with info extracted from "nr_description"
    df["gene_name"] = df["nr_description"].apply(get_gene_name)

    # Add accession column with info extracted form "nr_description"
    df["accession"] = df["nr_description"].apply(get_accession)

    # Add (n) extension to duplicate gene names
    dups = df[df.duplicated(subset="gene_name")]
    count = Counter() # Tracks the number of instances of each duplicate
    for index, row in dups.iterrows():
        count.update([str(row.gene_name)])
        # New name appends (n) to original name with n being the number of instances observed thus far
        new_name = (row.gene_name + " (" + (str(count[row.gene_name]) + ")"))
        # Update gene_name by index in the master DataFrame
        df.at[index, "gene_name"] = new_name

    # Rearange columns
    reorder_list = [   
                        "gene_id", "accession", "gene_name", "AS_1_FPKM", "AS_2_FPKM", "AS_3_FPKM", "AS_mean", "AS_stdev", 
                        "NS_1_FPKM", "NS_2_FPKM", "NS_3_FPKM", "NS_mean", "NS_stdev", "YPD_1_FPKM", "YPD_2_FPKM", 
                        "YPD_3_FPKM", "YPD_mean", "YPD_stdev", "YA_L2FC", "YN_L2FC", "AN_L2FC", "Cellular Component", "mol_func", 
                        "bio_process", "kegg_orth", "nr_description"
                   ]

    return df[reorder_list]

def get_gene_name(str_):
    # First regex catches most gene names
    re_1  = re.compile(r".+//(.+)\s+\[")
    # Second regex catches all others that are not "NaN"
    re_2 = re.compile(r"Full=([^;]+)")
    re_hit_1 = re.search(re_1, str_)
    if re_hit_1 is not None:
        return re_hit_1.group(1)

    re_hit_2 = re.search(re_2, str_)
    if re_hit_2 is not None:
        return re_hit_2.group(1)
    
    # Return "NaN" if no gene name found
    return "NaN"

def get_accession(str_):
    re_1  = re.compile(r"(.+)//")
    re_hit_1 = re.search(re_1, str_)
    if re_hit_1 is not None:
        return re_hit_1.group(1)
    
    # Return "NaN" if no accession number found
    return "NaN"

def search_df(df, _terms, locs, global_search=False, restrict=False):
    # Excludes hits to search terms prefixed with "non-"
    # Warning: this regex will exclude strings if the search term is at the immediate beginning
    _terms = ["[^(non\-)]" + term for term in _terms]

    if _terms is None or len(_terms) == 0:
        df["search_hit"] = True
        return df
    terms = "|".join(_terms)

    # Search all columns
    if global_search:
        locs = df.columns

    # Check if all provided locations are valid
    for col in locs:
        if col not in df.columns:
            raise ValueError(("Invalid column provided: " + col))

    # Hits will be a Series of Boolean values indicating one or more search hit
    hits = pd.Series(np.zeros(df.shape[0], dtype=bool))
    for col in locs:
        print("col =", col)
        if df[col].dtype == "object":
            sub_hits = df[col].str.lower().str.contains(terms)
            hits = (hits == True) | (sub_hits == True)

    df["search_hit"] = hits

    # If restrict, df is purged of rows without a search hit (True Boolean in hits)
    if restrict:
        reduced = df[hits].reset_index(drop=True)
        return reduced

    return df

def apply_thresh(df, col, lower=None, upper=None, quantile=False):
    # Allows slicing of data by quantile
    if quantile:
        lower = df[col].quantile(lower) if lower is not None else float("-inf")
        upper = df[col].quantile(upper) if upper is not None else float("inf")
    else:
        lower = lower if lower is not None else float("-inf")
        upper = upper if upper is not None else float("inf")

    return df[(df[col] > lower) & (df[col] < upper)]

def get_rna_df(args):
    df = pd.read_excel(args.rna_in_path)
    df = clean_rna_df(df, args)
    # df = apply_thresh(df, "AN_L2FC", upper = 0.01, quantile=True)
    df = search_df(df, args.terms, args.search_locs, global_search=args.universal, restrict=args.restrict)
    return df
    


In [135]:
# Set arguments for a given run
df_args = ArgBin(
                        rna_in_path_=Path(Path.cwd().parent / "in_files" / "rna_seq_input.xlsx"),
                        prot_in_path_=Path(Path.cwd().parent / "in_files" / "silac_input.xlsx"),
                        pair_in_path_=Path(Path.cwd().parent / "in_files" / "id_dump.csv"),
                        rna_out_path_=Path(Path.cwd().parent / "DataFrames" / "rna_seq_df.xlsx"),
                        prot_out_path_=Path(Path.cwd().parent / "DataFrames" / "prot_df.xlsx"),
                        comb_out_path_=Path(Path.cwd().parent / "DataFrames" / "comb_df.xlsx"),
                        sample_=None,
                        terms_=None,
                        search_locs_="nr_description", 
                        universal_=False,
                        dump_=False,
                        restrict_=False
                    )

In [136]:
# Generate RNA DataFrame and output to excel file
rna_df = get_rna_df(df_args)
rna_df.to_excel(df_args.rna_out_path)

## Generate SILAC DataFrame

The excel file containing the SILAC results are parsed, cleaned and then restricted based on parameters provided by the user. A DataFrame containing the parsed SILAC data is returned.

In [137]:
def clean_prot_df(df):
    # Fill empty cells with placeholder
    df.fillna("NaN", inplace=True)
    
    return df

def get_prot_df(args):
    df = pd.read_excel(args.prot_in_path)
    df = clean_prot_df(df)
    return df

In [138]:
# Generate Protein (SILAC) DataFrame and output to excel file
prot_df = get_prot_df(df_args)
prot_df.to_excel(df_args.prot_out_path)

## Merge RNA and SILAC DataFrames
The script "get_acc_uniprot_pairs.py" has already been run to generate a txt file containing Uniprot ID-GenBank accession number pairs. These ID pairs will be used to merged associated rows from the previously constructed RNA and protein DataFrames.

In [139]:
# Create dictionary from Uniprot ID: accession pairs
accs = []
uids = []
names = []

with open(df_args.pair_in_path, "r") as pair_file:
    for line in pair_file.readlines():
        info = line.strip().split(",")
        accs.append(info[0].strip())
        uids.append(info[1].strip())
        names.append(info[2].strip())
        
    # {Uniprot ID: accession number}
    uid_acc_dict = {uid:acc for uid, acc in zip(uids, accs)}
    # {accession number: gene name}
    acc_name_dict = {acc:name for acc, name in zip(accs, names)}

In [140]:
# Remove SILAC rows with no corresponding accession number
prot_df = prot_df.loc[prot_df["Majority protein IDs"].isin(uids)]
# Append accession numbers to protein DataFrame
prot_df.loc[:, "accession"] = prot_df["Majority protein IDs"].apply(lambda uid: uid_acc_dict[uid])

## Append RNA seq DataFrame row to corresponding protein DataFrame by matching accessions

In [141]:
# Remove accessions not found in protein DataFrame
acc_series = pd.Series(accs)
filt_1 = acc_series.isin(prot_df["accession"])
acc_series = acc_series.loc[filt_1]

# Remove accessions not found in RNA seq DataFrame or that occur more than once in the RNA seq DataFrame
filt_2 = acc_series.isin(rna_df["accession"].drop_duplicates(keep=False))
acc_series = acc_series.loc[filt_2]

# Restrict DataFrames to those with accessions matching one in refined accs Series
print("rna_df size before =", rna_df.shape)
print("prot_df size before =", prot_df.shape)

rna_df = rna_df.loc[rna_df["accession"].isin(acc_series)]
prot_df = prot_df.loc[prot_df["accession"].isin(acc_series)]

print("rna_df size after =", rna_df.shape)
print("prot_df size after =", prot_df.shape)

rna_df size before = (5988, 27)
prot_df size before = (4449, 26)
rna_df size after = (4442, 27)
prot_df size after = (4442, 26)


In [142]:
# Sort both DataFrames
rna_df = rna_df.sort_values(by="accession", axis=0).reset_index(drop=True)
prot_df = prot_df.sort_values(by="accession", axis=0).reset_index(drop=True)

# Prevents duplicate accession column
prot_df.drop(columns="accession", inplace=True)

# Stack horizontally
comb_df = pd.concat([rna_df, prot_df], sort=False, axis=1)

# Output raw merged DataFame
path_plus_raw = df_args.comb_out_path.parent / (df_args.comb_out_path.stem + "_raw" + df_args.comb_out_path.suffix)
comb_df.to_excel(path_plus_raw)

In [143]:
# Sets gene name from dictionary constructed from rows of id_dump.csv
comb_df["gene"] = comb_df["accession"].apply(lambda acc: acc_name_dict[acc])

In [144]:
# Delete unnecessary columns
drop_cols = [
                "AS_1_FPKM", "AS_2_FPKM", "AS_3_FPKM", "NS_1_FPKM", "NS_2_FPKM", "NS_3_FPKM", "YPD_1_FPKM", 
                "YPD_2_FPKM", "YPD_3_FPKM", "Peptides", "Unique peptides", "Sequence coverage [%]",
                "Unique sequence coverage [%]", "Mol. weight [kDa]", "Intensity", "Intensity L", "Intensity M", 
                "Intensity H", "Protein IDs", "Gene names"
            ]

# Drop all columns labeled "Unnamed X"
unnamed = [col for col in comb_df.columns if "Unnamed" in col]
drop_cols += unnamed

comb_df.drop(columns=drop_cols, inplace=True)

In [145]:
# Rename columns
comb_df = comb_df.rename(columns={
                                    "gene": "gene","gene_id": "transcript_id", "gene_name": "extended_name_rna", "AS_mean": "AS_rna_mean", 
                                    "AS_stdev": "AS_rna_stdev", "NS_mean": "NS_rna_mean", "NS_stdev": "NS_rna_stdev", "YPD_mean": "YPD_rna_mean", 
                                    "YPD_stdev": "YPD_rna_stdev", "YA_L2FC": "YA_rna_L2FC", "YN_L2FC": "YN_rna_L2FC", "AN_L2FC": "AN_rna_L2FC", 
                                    "Cellular Component": "cellular_component", "mol_func": "function", "SD-N/SD_1h": "SD-N/SD_1h_prot", 
                                    "SD-N/SD_6h": "SD-N/SD_6h_prot", "SD-AA/SD_1h": "SD-AA/SD_1h_prot", "SD-AA/SD_6h": "SD-AA/SD_6h_prot", 
                                    "SD-AA/SD-N_1h": "SD-AA/SD-N_1h_prot", "SD-AA/SD-N_6h": "SD-AA/SD-N_6h_prot", "Razor + unique peptides": "total_peptides", 
                                    "Unique + razor sequence coverage [%]": "total_percent_sequence_coverage", "Q-value": "q-value", 
                                    "Score": "score", "Majority protein IDs": "major_prot_id", "Protein names": "extended_name_prot", 
                                 }
                        )

In [146]:
# Reorder columns
comb_df = comb_df[[
           "gene", "extended_name_rna", "extended_name_prot", "NS_rna_mean", "NS_rna_stdev", "AS_rna_mean",
           "AS_rna_stdev", "YPD_rna_mean", "YPD_rna_stdev", "YN_rna_L2FC", "YA_rna_L2FC", "AN_rna_L2FC",
           "SD-N/SD_1h_prot", "SD-N/SD_6h_prot", "SD-AA/SD_1h_prot", "SD-AA/SD_6h_prot", "SD-AA/SD-N_1h_prot",
           "SD-AA/SD-N_6h_prot", "total_peptides", "total_percent_sequence_coverage", "q-value", "score", 
           "transcript_id", "accession", "major_prot_id", "nr_description", "cellular_component", "function", 
           "bio_process", "kegg_orth", "search_hit",

        ]]

In [147]:
# Set index to gene name
comb_df.set_index("gene", inplace=True)

In [148]:
# Output cleaned DataFrame to excel file
comb_df.to_excel(df_args.comb_out_path)